In [ ]:
# Celda 1: Configuración y carga de datos
import sys
sys.path.append('../src')

import numpy as np
import matplotlib.pyplot as plt
from data_preprocessing import MovieDataPreprocessor
from cnn_model import CNNModel
from mlp_model import MLPModel
from utils import setup_gpu, plot_training_history

setup_gpu()

# Cargar y preparar datos
DATA_PATH = "../data/raw"
preprocessor = MovieDataPreprocessor(DATA_PATH)
X, y_labels = preprocessor.load_data()
y_categorical, y_encoded = preprocessor.preprocess_labels(y_labels)

# División de datos
X_train, X_val, X_test, y_train, y_val, y_test = preprocessor.create_data_splits(X, y_categorical)

print(f"Datos de entrenamiento: {X_train.shape}")
print(f"Datos de validación: {X_val.shape}")
print(f"Datos de prueba: {X_test.shape}")

# Celda 2: Construcción del modelo CNN
print("🏗️ CONSTRUCCIÓN DEL MODELO CNN")
print("="*40)

cnn_model = CNNModel(input_shape=(224, 224, 3), num_classes=preprocessor.num_classes)
cnn = cnn_model.build_model(architecture='custom')

# Mostrar arquitectura
cnn.summary()

from tensorflow.keras.utils import plot_model
plot_model(cnn, to_file='../results/plots/cnn_architecture.png', 
           show_shapes=True, show_layer_names=True, rankdir='TB', dpi=300)

# Celda 3: Entrenamiento CNN
print("🚀 ENTRENAMIENTO CNN")
print("="*25)

# Crear generadores de datos
train_gen, val_gen = preprocessor.create_data_generators(X_train, y_train, X_val, y_val)

# Entrenar modelo
cnn_callbacks = cnn_model.get_callbacks('cnn_notebook')
cnn_history = cnn_model.train(train_gen, val_gen, epochs=50, callbacks=cnn_callbacks)

# Visualizar entrenamiento
plot_training_history(cnn_history, 'CNN', '../results/plots')

# Celda 4: Construcción del modelo MLP
print("🏗️ CONSTRUCCIÓN DEL MODELO MLP")
print("="*40)

mlp_model = MLPModel(input_shape=(224, 224, 3), num_classes=preprocessor.num_classes)
mlp = mlp_model.build_model(hidden_layers=[2048, 1024, 512], dropout_rates=[0.6, 0.5, 0.4])

# Mostrar arquitectura
mlp.summary()

plot_model(mlp, to_file='../results/plots/mlp_architecture.png', 
           show_shapes=True, show_layer_names=True, rankdir='TB', dpi=300)

# Celda 5: Entrenamiento MLP
print("🚀 ENTRENAMIENTO MLP")
print("="*25)

mlp_callbacks = mlp_model.get_callbacks('mlp_notebook')
mlp_history = mlp_model.train(X_train, y_train, X_val, y_val, 
                              epochs=100, batch_size=32, callbacks=mlp_callbacks)

# Visualizar entrenamiento
plot_training_history(mlp_history, 'MLP', '../results/plots')

# Celda 6: Comparación de entrenamientos
def compare_training_histories(cnn_hist, mlp_hist):
    """Comparar historiales de entrenamiento"""
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Loss de entrenamiento
    axes[0,0].plot(cnn_hist.history['loss'], label='CNN', color='blue')
    axes[0,0].plot(mlp_hist.history['loss'], label='MLP', color='red')
    axes[0,0].set_title('Training Loss Comparison')
    axes[0,0].set_xlabel('Epochs')
    axes[0,0].set_ylabel('Loss')
    axes[0,0].legend()
    axes[0,0].grid(True, alpha=0.3)
    
    # Loss de validación
    axes[0,1].plot(cnn_hist.history['val_loss'], label='CNN', color='blue')
    axes[0,1].plot(mlp_hist.history['val_loss'], label='MLP', color='red')
    axes[0,1].set_title('Validation Loss Comparison')
    axes[0,1].set_xlabel('Epochs')
    axes[0,1].set_ylabel('Loss')
    axes[0,1].legend()
    axes[0,1].grid(True, alpha=0.3)
    
    # Accuracy de entrenamiento
    axes[1,0].plot(cnn_hist.history['accuracy'], label='CNN', color='blue')
    axes[1,0].plot(mlp_hist.history['accuracy'], label='MLP', color='red')
    axes[1,0].set_title('Training Accuracy Comparison')
    axes[1,0].set_xlabel('Epochs')
    axes[1,0].set_ylabel('Accuracy')
    axes[1,0].legend()
    axes[1,0].grid(True, alpha=0.3)
    
    # Accuracy de validación
    axes[1,1].plot(cnn_hist.history['val_accuracy'], label='CNN', color='blue')
    axes[1,1].plot(mlp_hist.history['val_accuracy'], label='MLP', color='red')
    axes[1,1].set_title('Validation Accuracy Comparison')
    axes[1,1].set_xlabel('Epochs')
    axes[1,1].set_ylabel('Accuracy')
    axes[1,1].legend()
    axes[1,1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('../results/plots/training_comparison.png', dpi=300, bbox_inches='tight')
    plt.show()

compare_training_histories(cnn_history, mlp_history)